# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] >= 30) & (city_data_df["Cloudiness"] <= 50) & (city_data_df["Humidity"] <= 70),:]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna(how="any")

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
98,98,itarema,-2.9248,-39.9167,31.07,54,14,9.48,BR,1666108293
162,162,sao miguel do araguaia,-13.2750,-50.1628,40.09,13,33,3.87,BR,1666108241
186,186,upington,-28.4478,21.2561,32.21,19,20,3.09,ZA,1666108351
253,253,tchaourou,8.8865,2.5975,31.43,56,37,1.28,BJ,1666108386
289,289,san andres,12.5847,-81.7006,30.01,70,40,3.09,CO,1666108406
300,300,aquiraz,-3.9014,-38.3911,31.92,56,22,7.50,BR,1666108415
316,316,bathsheba,13.2167,-59.5167,31.34,70,40,6.17,BB,1666108423


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,trairi,BR,-3.2778,-39.2689,51,
22,guiratinga,BR,-16.3494,-53.7583,14,
29,saint-louis,SN,16.3333,-15.0000,24,
98,itarema,BR,-2.9248,-39.9167,54,
162,sao miguel do araguaia,BR,-13.2750,-50.1628,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "limit": 10,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
trairi - nearest hotel: Pousada Villa Aurora
guiratinga - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
itarema - nearest hotel: Pousada Oásis
sao miguel do araguaia - nearest hotel: Hotel São Miguel
upington - nearest hotel: Die Eiland Holiday Ressort
tchaourou - nearest hotel: Auberge
san andres - nearest hotel: Hotel Decameron Isleño
aquiraz - nearest hotel: Beach Spa Hotel
bathsheba - nearest hotel: Atlantis Hotel
bogande - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
tessalit - nearest hotel: No hotel found
the valley - nearest hotel: Ceblue Luxury Villas
caiaponia - nearest hotel: No hotel found
abu zabad - nearest hotel: No hotel found
brokopondo - nearest hotel: Matu eiland
sawakin - nearest hotel: Hotel Halfa
nouadhibou - nearest hotel: Hotel valencia
bilma - nearest hotel: No hotel found
road town - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,trairi,BR,-3.2778,-39.2689,51,Pousada Villa Aurora
22,guiratinga,BR,-16.3494,-53.7583,14,No hotel found
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
98,itarema,BR,-2.9248,-39.9167,54,Pousada Oásis
162,sao miguel do araguaia,BR,-13.2750,-50.1628,13,Hotel São Miguel
186,upington,ZA,-28.4478,21.2561,19,Die Eiland Holiday Ressort
253,tchaourou,BJ,8.8865,2.5975,56,Auberge
289,san andres,CO,12.5847,-81.7006,70,Hotel Decameron Isleño
300,aquiraz,BR,-3.9014,-38.3911,56,Beach Spa Hotel
316,bathsheba,BB,13.2167,-59.5167,70,Atlantis Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)